# 2D Grid Scan at LCLS

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = str(6)

In [ ]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../")
    sys.path.append("../")

run_dir = "/home/physics/ml_tuning/20230904_LCLS_Injector/"
print(sys.path)

## Set up image diagnostic

In [ ]:
from scripts.image import ImageDiagnostic
import yaml

fname = "../OTR3_config.yml"
image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
image_diagnostic.background_file = run_dir + "OTRS_IN20_621_background.npy"
print(image_diagnostic.yaml())

In [ ]:
#image_diagnostic.measure_background(file_location=run_dir)

In [ ]:
image_diagnostic.background_file

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_diagnostic.background_image)

In [ ]:
image_diagnostic.test_measurement()

## Define VOCS

In [ ]:
import pandas as pd
from xopt import VOCS

In [ ]:
filename = "../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')

# # replace w turbo ranges
# VARIABLE_RANGES = {"QUAD:IN20:121:BCTRL": [-0.010216134865221826, 0.0004946549130899012],
# "QUAD:IN20:122:BCTRL": [-0.0037497838633867926, 0.0046256696462631235],
# "QUAD:IN20:361:BCTRL": [-3.566282173921773, -2.952950549721718],
# "QUAD:IN20:371:BCTRL": [2.1990287090002765, 2.942573140353014],
# "QUAD:IN20:425:BCTRL": [-1.8677221410349012, -1.08],
# "QUAD:IN20:441:BCTRL": [-1.08, 0.3188352971091497],
# "QUAD:IN20:511:BCTRL": [2.5533611167764256, 4.642105910740238],
# "QUAD:IN20:525:BCTRL": [-4.004095742514585, -2.429185827568884],
# "SOLN:IN20:121:BCTRL": [0.4603330888274317, 0.4898202648048277]}

IMAGE_CONSTRAINTS = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", image_diagnostic.min_log_intensity]
}

In [ ]:
VARIABLES = ["SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL"]
# VARIABLES = ["QUAD:IN20:121:BCTRL", "QUAD:IN20:122:BCTRL"]
# VARIABLES = ["QUAD:IN20:361:BCTRL", "QUAD:IN20:371:BCTRL"]

# VARIABLES = ["SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL", "QUAD:IN20:122:BCTRL",
#              "QUAD:IN20:361:BCTRL", "QUAD:IN20:371:BCTRL", "QUAD:IN20:425:BCTRL",
#              "QUAD:IN20:441:BCTRL", "QUAD:IN20:511:BCTRL", "QUAD:IN20:525:BCTRL"]

vocs = VOCS(
    variables = {ele: VARIABLE_RANGES[ele] for ele in VARIABLES},
    constraints = IMAGE_CONSTRAINTS,
    objectives = {"total_size": "MINIMIZE"},
)
print(vocs.as_yaml())

## Update variable ranges to full model domain

In [ ]:
model_path = "lcls_cu_injector_nn_model/"
input_variables, output_variables = variables_from_yaml(open(model_path + "model/pv_variables.yml"))
input_variables = {name.replace("BACT", "BCTRL"): ele for name, ele in input_variables.items()}

vocs.variables = {k: input_variables[k].value_range for k in vocs.variable_names}
print(vocs.as_yaml())

## Define evaluate function

In [ ]:
from time import sleep

import torch
import numpy as np
from epics import caput, caget_many

In [ ]:
def eval_beamsize(input_dict, generator = None):
    global image_diagnostic
    # set PVs
    for k, v in input_dict.items():
        print(f'CAPUT {k} {v}')
        caput(k, v)

    sleep(2.0)

    # get beam sizes from image diagnostic
    metadata = input_dict
    results = image_diagnostic.measure_beamsize(1, **metadata)
    results["S_x_mm"] = np.array(results["Sx"]) * 1e-3
    results["S_y_mm"] = np.array(results["Sy"]) * 1e-3

    # get other PV's NOTE: Measurements not synchronous with beamsize measurements!
    results = results

    # add total beam size
    results["total_size"] = np.sqrt(np.array(results["Sx"]) ** 2 + np.array(results["Sy"]) ** 2)
    # results["total_size"] = np.sqrt(np.abs(np.array(results["Sx"])) * np.array(results["Sy"]))
    
    return results

## Run Xopt

In [ ]:
from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators import UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

# remember to set use low noise prior to false!!!
model_constructor = StandardModelConstructor(
    use_low_noise_prior=False,
)
generator = UpperConfidenceBoundGenerator(
    vocs=vocs,
    model_constructor=model_constructor,
    # turbo_controller="optimize"
)
generator.numerical_optimizer.max_iter = 200
evaluator = Evaluator(function=eval_beamsize, function_kwargs={"generator": generator})
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
#X.options.dump_file = run_dir + "grid_scan_2d.yml"
X

In [ ]:
# generate input mesh
n_grid = 10
x_lim = torch.tensor([X.vocs.variables[k] for k in X.vocs.variable_names])
x_i = [torch.linspace(*x_lim[i], n_grid) for i in range(x_lim.shape[0])]
x_mesh = torch.meshgrid(*x_i, indexing="ij")
x = torch.hstack([ele.reshape(-1, 1) for ele in x_mesh]).double()

In [ ]:
# evaluate input mesh
X.evaluate_data(pd.DataFrame({k: x[:, X.vocs.variable_names.index(k)] for k in X.vocs.variable_names}))

In [ ]:
X.data.plot(y="total_size")

In [ ]:
X.data.plot(y=X.vocs.variable_names)